In [6]:
import prosperity3bt
import pandas as pd
import itertools
import numpy as np
from tqdm import tqdm

from prosperity3bt.runner import run_backtest
from prosperity3bt.models import TradeMatchingMode
from prosperity3bt.file_reader import FileSystemReader
from pathlib import Path

from importlib import reload

In [4]:
def day_profit(result):
    last_timestamp = result.activity_logs[-1].timestamp

    profit = 0
    for row in reversed(result.activity_logs):
        if row.timestamp != last_timestamp:
            break

        profit += row.columns[-1]

    return profit

def backtest_days(trader, days=[-2, -1, 0], data_path='.', round_num=1):
    profits = []
    for day in days:
        backtest_results = prosperity3bt.runner.run_backtest(trader, file_reader=FileSystemReader(Path(data_path)), round_num=round_num, day_num=day, print_output=False, trade_matching_mode=TradeMatchingMode.all, no_names=True, show_progress_bar=False)
        profit = day_profit(backtest_results)
        profits.append(profit)
    return profits

def generate_param_combinations(param_grid):
    param_names = param_grid.keys()
    param_values = param_grid.values()
    combinations = list(itertools.product(*param_values))
    return [dict(zip(param_names, combination)) for combination in combinations]

In [7]:
import ink_trader

reload(ink_trader)

param_grid = {
    # 'ink_change_threshold_pct': [0.01, 0.012, 0.015, 0.018, 0.02, 0.022, 0.025, 0.03],
    'ink_change_threshold_pct': [0.01, 0.012, 0.015, 0.018, 0.02, 0.022],
    'ink_window_size': [3, 5, 6, 8, 10, 12, 15, 20, 25],
    'ink_position_limit': [30, 40, 50],
    # 'ink_position_limit': [30, 35, 40, 45, 50],
    'clear_price_thresh': [0, 1, 2, 3, 5]
}

PARAMS = {
    'ink_change_threshold_pct': 0.02,
    'ink_window_size': 10,
    'ink_position_limit': 50,
}

param_profits = []

for params in tqdm(generate_param_combinations(param_grid)):
    trader = ink_trader.Trader(params)
    days = [0]
    profits = backtest_days(trader, days=days, data_path='.', round_num=1)
    for d in days:
        params[f'profit_day_{d}'] = profits[days.index(d)]
    param_profits.append(params)

100%|██████████| 810/810 [24:14<00:00,  1.80s/it]    


In [8]:
df = pd.DataFrame(param_profits)
df['tot_profit'] = df[[f'profit_day_{d}' for d in days]].sum(axis=1)
df.sort_values(by='tot_profit', ascending=False, inplace=True)
df

,ink_change_threshold_pct,ink_window_size,ink_position_limit,clear_price_thresh,profit_day_0,tot_profit
250,0.012,20,50,0,903.0,903.0
400,0.015,25,50,0,903.0,903.0
251,0.012,20,50,1,778.0,778.0
252,0.012,20,50,2,778.0,778.0
401,0.015,25,50,1,778.0,778.0
...,...,...,...,...,...,...
360,0.015,15,30,0,0.0,0.0
361,0.015,15,30,1,0.0,0.0
362,0.015,15,30,2,0.0,0.0
363,0.015,15,30,3,0.0,0.0


In [11]:
df

,ink_change_threshold_pct,ink_window_size,ink_position_limit,clear_price_thresh,profit_day_0,tot_profit
250,0.012,20,50,0,903.0,903.0
400,0.015,25,50,0,903.0,903.0
251,0.012,20,50,1,778.0,778.0
252,0.012,20,50,2,778.0,778.0
401,0.015,25,50,1,778.0,778.0
...,...,...,...,...,...,...
360,0.015,15,30,0,0.0,0.0
361,0.015,15,30,1,0.0,0.0
362,0.015,15,30,2,0.0,0.0
363,0.015,15,30,3,0.0,0.0


In [12]:
profits

[0.0]

In [ ]:
df.to_csv('backtests/inktrader_bt_results.csv')

,ink_change_threshold_pct,ink_window_size,profit_day_-2,profit_day_-1,profit_day_0,tot_profit
0,0.02,5,4273.0,3450.0,0.0,7723.0
1,0.02,10,4121.0,3450.0,0.0,7571.0
2,0.02,15,4121.0,3450.0,0.0,7571.0
3,0.03,5,0.0,3450.0,0.0,3450.0
4,0.03,10,0.0,3450.0,0.0,3450.0
5,0.03,15,0.0,3450.0,0.0,3450.0
6,0.04,5,0.0,3450.0,0.0,3450.0
7,0.04,10,0.0,3450.0,0.0,3450.0
8,0.04,15,0.0,3450.0,0.0,3450.0
